In [73]:
from gensim.parsing.porter import PorterStemmer
from gensim.parsing.preprocessing import remove_stopwords
from gensim import corpora
from gensim import models
from gensim import similarities
from smart_open import open

In [74]:
#Gensim trae sus propias funciones para el procesamiento de texto
text_test = "the runner ran on the running trail for their runs"

print(remove_stopwords(text_test))
p = PorterStemmer()
print(p.stem_sentence(text_test))

runner ran running trail runs
the runner ran on the run trail for their run


<h2>Corpus Streaming</h2>
Normalmente los corpus reciden completamente en la memoria. Supongamos que hay millones de documentos en el corpus. Almacenarlos todos en RAM no será suficiente. En su lugar, supongamos que los documentos se almacenan en un archivo en el disco, un documento por línea. Gensim puede procesar un documento a la vez y actualizar dinamicamente e calculo de IDF y del vocabulario.

In [75]:
#Procesamiento de un texto utilizando las funciones de Gensim
def process(text):
    doc_nor= text.lower()
    doc_sw = remove_stopwords(doc_nor)
    doc_stem = p.stem_sentence(doc_sw)
    return doc_stem.split()

In [77]:
#Creaccion de lista con documentos procesados

docDict = []
for line in open('DocsParaDiccionario.txt', 'rb'):
    docDict.append(process(line))

print(docDict[0]) #Documentos ya procesados, de esta forma deben ser enviados al metodo de Gensim

['amazon', 'distribut', 'download', 'stream', 'video,', 'music,', 'audiobook', 'amazon', 'prime', 'video,', 'amazon', 'music,', 'audibl', 'subsidiaries.', 'amazon', 'publish', 'arm,', 'amazon', 'publishing,', 'film', 'televis', 'studio,', 'amazon', 'studios,', 'cloud', 'comput', 'subsidiary,', 'amazon', 'web', 'services.']


In [78]:
#Ahora si llamamos al metodo de Gensim para crear el diccionario a partir de los documentos
dictionary = corpora.Dictionary(docDict)
dictionary.save('clase3.dict')
print(dictionary)

#Notese que Gensim de una vez asigna a cada token un id
print(dictionary.token2id)

Dictionary(309 unique tokens: ['amazon', 'arm,', 'audibl', 'audiobook', 'cloud']...)
{'amazon': 0, 'arm,': 1, 'audibl': 2, 'audiobook': 3, 'cloud': 4, 'comput': 5, 'distribut': 6, 'download': 7, 'film': 8, 'music,': 9, 'prime': 10, 'publish': 11, 'publishing,': 12, 'services.': 13, 'stream': 14, 'studio,': 15, 'studios,': 16, 'subsidiaries.': 17, 'subsidiary,': 18, 'televis': 19, 'video,': 20, 'web': 21, 'american': 22, 'best': 23, 'browsers.': 24, 'compani': 25, 'computers,': 26, 'consum': 27, 'corpor': 28, 'develops,': 29, 'edg': 30, 'electronics,': 31, 'explor': 32, 'headquart': 33, 'internet': 34, 'known': 35, 'licenses,': 36, 'line': 37, 'manufactures,': 38, 'microsoft': 39, 'multin': 40, 'offic': 41, 'oper': 42, 'person': 43, 'product': 44, 'redmond,': 45, 'relat': 46, 'sell': 47, 'softwar': 48, 'software,': 49, 'suite,': 50, 'support': 51, 'systems,': 52, 'technolog': 53, 'washington.': 54, 'window': 55, '14': 56, '1998': 57, '1998.': 58, '4,': 59, '56': 60, 'brin': 61, 'califor

In [79]:
new_doc = "Human media interaction media"
new_doc_bow = dictionary.doc2bow(process(new_doc))
print(new_doc_bow)  # ¿Si hay cuatro palabras pq solo dos tokens?

[(135, 1), (288, 2)]


In [80]:
#Construccion del corpus == Cada documento representado con el vocabulario/diccionario definido anteriormente (Bolsa de palabras)
class MyCorpus(object):
    def __iter__(self):
        for line in open('mycorpus.txt', 'rb'):
            yield dictionary.doc2bow(process(line))

corpus_memory_friendly = MyCorpus()#Todos mis documentos ahora estan representados como una bolsa de palabras
#Almaceno mi corpus
corpora.MmCorpus.serialize('corpus.mm', corpus_memory_friendly)  

In [81]:
#Cargo mi corpus
corpus = corpora.MmCorpus('corpus.mm')
print(corpus) #No lo carga en memoria
#Para leer la representacion de bolda de palabras resultante de cada documento en el corpus
for doc in corpus:
    print(doc)

MmCorpus(12 documents, 309 features, 375 non-zero entries)
[(0, 7.0), (1, 1.0), (2, 1.0), (3, 1.0), (4, 1.0), (5, 1.0), (6, 1.0), (7, 1.0), (8, 1.0), (9, 2.0), (10, 1.0), (11, 1.0), (12, 1.0), (13, 1.0), (14, 1.0), (15, 1.0), (16, 1.0), (17, 1.0), (18, 1.0), (19, 1.0), (20, 2.0), (21, 1.0)]
[(13, 1.0), (21, 1.0), (22, 1.0), (23, 1.0), (24, 1.0), (25, 1.0), (26, 1.0), (27, 1.0), (28, 1.0), (29, 1.0), (30, 1.0), (31, 1.0), (32, 1.0), (33, 1.0), (34, 1.0), (35, 1.0), (36, 1.0), (37, 1.0), (38, 1.0), (39, 3.0), (40, 1.0), (41, 1.0), (42, 1.0), (43, 1.0), (44, 1.0), (45, 1.0), (46, 1.0), (47, 1.0), (48, 1.0), (49, 1.0), (50, 1.0), (51, 1.0), (52, 1.0), (53, 1.0), (54, 1.0), (55, 1.0)]
[(25, 1.0), (56, 1.0), (57, 1.0), (58, 1.0), (59, 1.0), (60, 1.0), (61, 1.0), (62, 1.0), (63, 1.0), (64, 1.0), (65, 2.0), (66, 1.0), (67, 1.0), (68, 1.0), (69, 1.0), (70, 2.0), (71, 1.0), (72, 1.0), (73, 1.0), (74, 1.0), (75, 1.0), (76, 1.0), (77, 1.0), (78, 1.0), (79, 1.0), (80, 1.0), (81, 1.0), (82, 1.0), (8

<h2>Vector Space Model - TF-IDF</h2>

In [82]:
#Suponga que tenemos el diccionario y corpus en disco
#Primero necesitamos cargarlos
dictionary = corpora.Dictionary.load('clase3.dict')
corpus = corpora.MmCorpus('corpus.mm')

In [83]:
#Construyendo un modelo VSM con ponderacion por TF-IDF
tfidf = models.TfidfModel(corpus) #Inizializacion

In [84]:
#Supongamos que queremos calcular la similitud de un nuevo documento con los documentos en el corpus
#1. Construimos la representación vectorial del query
query_doc = "Amazon vs Microsoft"
query_doc_bow = dictionary.doc2bow(process(new_doc))
print(query_doc_bow)
print(tfidf[query_doc_bow])

[(135, 1), (288, 2)]
[(135, 0.447213595499958), (288, 0.894427190999916)]


In [85]:
#2. Ahora construimos la matrix t/d con los documentos contra los cuales queremos compara el query
index = similarities.MatrixSimilarity(tfidf[corpus]) 
# Vamos a salvar el inidice resultante para no tener que recalcularlo cada vez que los necesitemos.
index.save('clase3tfidf.index')

In [86]:
#Una vez guardado lo podemos cargar sin necesidad de recalcularlo
index = similarities.MatrixSimilarity.load('clase3tfidf.index')

In [87]:
# Ejecutar las similitudes entre el documento query y el corpus ahora es muy facil!!!
sims = index[tfidf[query_doc_bow]]
print(list(enumerate(sims)))  # print (document_number, document_similarity) 2-tuples

[(0, 0.0), (1, 0.0), (2, 0.0), (3, 0.0), (4, 0.14518946), (5, 0.0), (6, 0.0), (7, 0.0), (8, 0.0), (9, 0.0), (10, 0.0), (11, 0.07944595)]
